In [2]:
import duckdb
import pandas as pd

In [107]:
DeliveryData = pd.DataFrame({
    'delivery_id': [1, 2, 3, 4,5],
    'date': ["2019-08-10", "2019-06-05", "2019-08-01", "2019-08-01", "2019-02-01"],
    'store_id': [123, 345, 123, 456, 234],
    "order_value": [12.7, 30.9, 55.4, 22.2, 10]
})

DeliveryData

,delivery_id,date,store_id,order_value
0,1,2019-08-10,123,12.7
1,2,2019-06-05,345,30.9
2,3,2019-08-01,123,55.4
3,4,2019-08-01,456,22.2
4,5,2019-02-01,234,10.0


In [104]:
sql = """
SELECT store_id, SUBSTRING(date,1,7) as YearMth, COUNT(delivery_id) as deliveries, SUM(order_value) as Revenue
FROM DeliveryData
GROUP BY YearMth, store_id
HAVING YearMth = '2019-08'
and store_id = 123
"""

duckdb.sql(sql)

┌──────────┬─────────┬────────────┬─────────┐
│ store_id │ YearMth │ deliveries │ Revenue │
│  int64   │ varchar │   int64    │ double  │
├──────────┼─────────┼────────────┼─────────┤
│      123 │ 2019-08 │          2 │    68.1 │
└──────────┴─────────┴────────────┴─────────┘

In [ ]:
DeliveryData = pd.DataFrame({
    'delivery_id': [1, 2, 3, 4],
    'date': ["2019-08-10", "2019-06-05", "2019-08-01", "2019-01-01"],
    'store_id': [123, 345, 123, 456],
    "order_value": [12.7, 30.9, 55.4, 22.2]
})

How many stores are under business id 890? Of these stores, how many of them are on DashPass?

In [101]:
businessData = pd.DataFrame({
    'business_id': [890, 891, 456, 745, 890, 888, 1045],
    'business_name': ['Taco Shop', 'Fruit Shop', 'Tea Sop', 'Ice Cream Shop', 'Fruit', 'Casa', 'Bar'],
    'store_id': [123, 345, 678, 987, 456, 345, 234],
    'dashpass': ['Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes']
})
display(businessData)

,business_id,business_name,store_id,dashpass
0,890,Taco Shop,123,Yes
1,891,Fruit Shop,345,Yes
2,456,Tea Sop,678,No
3,745,Ice Cream Shop,987,No
4,890,Fruit,456,No
5,888,Casa,345,Yes
6,1045,Bar,234,Yes


In [78]:
sql = """
SELECT
  business_id,
  COUNT(DISTINCT(store_id)) as StoreCount,
  SUM(CASE
    WHEN dashpass = 'Yes' THEN 1
    ELSE 0
  END) AS DashPassCount
FROM businessData
WHERE business_id = 890
GROUP BY business_id
"""

duckdb.sql(sql)

┌─────────────┬────────────┬───────────────┐
│ business_id │ StoreCount │ DashPassCount │
│    int64    │   int64    │    int128     │
├─────────────┼────────────┼───────────────┤
│         890 │          2 │             1 │
└─────────────┴────────────┴───────────────┘

3. How many stores do Kevin and Carla manage?

In [79]:
bizmap = pd.DataFrame({
    'account_owner': ['Kevin', 'Matt', 'Kevin', 'Carla'],
    'business_id': [888, 4546, 890, 1045],
    'region_id': [1, 2, 4, 5]
})
display(bizmap)

,account_owner,business_id,region_id
0,Kevin,888,1
1,Matt,4546,2
2,Kevin,890,4
3,Carla,1045,5


In [80]:
sql = """
SELECT
  own.account_owner,
  COUNT(DISTINCT(biz.store_id)) as StoreCount
FROM bizmap own
JOIN businessData biz ON own.business_id = biz.business_id
WHERE own.account_owner IN ('Kevin', 'Carla')
GROUP BY own.account_owner
"""


duckdb.sql(sql)

┌───────────────┬────────────┐
│ account_owner │ StoreCount │
│    varchar    │   int64    │
├───────────────┼────────────┤
│ Kevin         │          3 │
│ Carla         │          1 │
└───────────────┴────────────┘

 Show monthly sales for stores that Kevin and Carla manage, including a running total of cumulative sales.

In [118]:
sql = """
WITH cte AS (
  SELECT
  own.account_owner,
  own.business_id,
  biz.store_id,
  SUBSTRING(del.date,1,7) as YearMth,
  SUM(del.order_value) as Revenue
FROM bizmap own
JOIN businessData biz ON own.business_id = biz.business_id
JOIN DeliveryData del ON biz.store_id = del.store_id
WHERE own.account_owner IN ('Kevin', 'Carla')
GROUP BY 1,2,3,4
ORDER BY 1,2,3,4 )

SELECT
  account_owner,
  YearMth,
  store_id,
  Revenue,
  SUM(Revenue) OVER (PARTITION BY YearMth ORDER BY store_id) as CumSum

FROM cte

"""


duckdb.sql(sql)

┌───────────────┬─────────┬──────────┬─────────┬────────┐
│ account_owner │ YearMth │ store_id │ Revenue │ CumSum │
│    varchar    │ varchar │  int64   │ double  │ double │
├───────────────┼─────────┼──────────┼─────────┼────────┤
│ Kevin         │ 2019-06 │      345 │    30.9 │   30.9 │
│ Kevin         │ 2019-08 │      123 │    68.1 │   68.1 │
│ Kevin         │ 2019-08 │      456 │    22.2 │   90.3 │
│ Carla         │ 2019-02 │      234 │    10.0 │   10.0 │
└───────────────┴─────────┴──────────┴─────────┴────────┘